# Discovering a parsimonious neural network - particle under an external non-linear potential #

<i>Saaketh Desai</i>, and <i>Alejandro Strachan</i>, School of Materials Engineering, Purdue University <br>

This notebook describes the procedure to train a parsimonious neural network, i.e., a network designed to reproduce the training and testing datasets in the simplest, most interpretable manner possible. We use Keras to train neural networks and the DEAP package for genetic algorithms. The outline of this notebook is:

1. Read datasets and split into training and testing sets
2. Create a generic model
3. Define the objective function for the genetic algorithm
4. Set up the genetic algorithm and save the results

In [ ]:
import tensorflow as tf
import keras
from keras import backend as K
from keras import initializers
from keras.layers import Dense, Input, Activation
from keras.models import Sequential, Model, load_model
from keras.layers.merge import add, concatenate

from sklearn.model_selection import train_test_split

import numpy as np
import sys
import os
import random

from deap import base, creator, tools, algorithms
from multiprocessing import Pool

We first define some globally relevant parameters and units

In [ ]:
#x = pm; t = fs => v = pm/fs; a = pm/fs^2
natoms = 2
mass = 26.982
timestep = 1.0
constant = 9.648532952214415e-1

## Step 1: Read datasets and split into training and testing sets
The training and testing data is generated using the LAMMPS software and is stored in the LAMMPS dump file format. The helper function below reads in the file paths to the LAMMPS dump files and then parses them into input and output arrays, where the inputs are position and velocity at time $t$ and the outputs are the position and velocity at time $t + \Delta t$. We then use the scikit-learn <i> train_test_split()</i> function to split the datasets into training and testing sets.

In [ ]:
def read_files(documents):
    input_storage = []; output_storage = []

    for file in documents:
        with open(file) as f:
            data = f.readlines()

        n=9+natoms
        data = data[n-2::n]
        data = [x.strip().split() for x in data] # First atom
        data = np.array([[float(item) for item in sublist] for sublist in data]) #Turn strings into floats

        x = data[:,1].reshape(len(data),1)*100
        vx = data[:,4].reshape(len(data),1)/10
        fx = (data[:,7]).reshape(len(data),1)

        data_xv = np.concatenate((x,vx,fx), axis=1)

        input_storage.append(data_xv[:-10:10,:])
        output_storage.append(data_xv[10::10,:])
    
    total_length_list = [len(i) for i in input_storage]
    total_length = sum(total_length_list)

    inputs = np.zeros((total_length, 3))
    outputs = np.zeros((total_length, 3))
    start = 0; end = 0
    for i in range(len(documents)):
        end += len(input_storage[i])
        inputs[start:end, :] = input_storage[i]
        outputs[start:end, :] = output_storage[i]
        start = end

    print("Inputs", inputs.shape)
    print("Outputs", outputs.shape)
    
    return inputs, outputs

In [ ]:
documents = ["../data/Al_3K_small.dump",
             "../data/Al_273K_small.dump",
             "../data/Al_530K_small.dump",
             "../data/Al_703K_small.dump"]
inputs, outputs = read_files(documents)

documents = ["../data/Al_300K_small.dump"]
test_inputs, test_outputs = read_files(documents)

train_inputs, val_inputs, train_outputs, val_outputs = train_test_split(inputs[:,0:2], outputs[:,0:2],
                                                                        test_size=0.2, random_state=0)

## Step 2: Create a generic model

We will first define variables relevant to setting up the model. We then read in the parameters for the pre-trained force model, keeping those weights fixed. The force model is trained outside of this notebook, and is a model that is trained to reproduce the force acting on the atom, given its raw position

In [ ]:
act_dict = {0: 'linear', 1: 'relu', 2: 'tanh', 3: 'elu'}
np.random.seed(100000)
weight_dict = {0: 0, 1: 0.5, 2: 1, 3: 2, 4: timestep/2, 5: timestep, 6: 2*timestep, 7: np.random.uniform(-1,1,1)[0]}
nact_terms = 6
nweight_terms = 20

In [ ]:
model = tf.keras.models.load_model('../data/force_model_small.h5')
w1 = model.layers[1].get_weights()
w2 = model.layers[2].get_weights()
w3 = model.layers[3].get_weights()

The next few functions help create a generic neural network, the parameters for which will be decided by the genetic algorithm. In the <i> create_model() </i> function, we set the weights and activations according to genes of the individual, as decided by the genetic algorithm

In [ ]:
def create_node(input1, input2, name, trainable1, trainable2, act):
    base = name
    n1 = base + "1"
    n2 = base + "2"
    an1 = Dense(1, activation = 'linear', use_bias = False, name=n1, trainable=trainable1) (input1)
    an2 = Dense(1, activation = 'linear', use_bias = False, name=n2, trainable=trainable2) (input2)
    an = add([an1, an2])
    an = Activation(act) (an)
    return an

In [ ]:
def create_model(x):
    bias_initial = keras.initializers.Zeros()

    trainable_list = []
    for i in range(nweight_terms):
        if (x[i+nact_terms] == 7):
            trainable_list.append(True)
        else:
            trainable_list.append(False)

    input_position = Input(shape=(1,))
    input_velocity = Input(shape=(1,))

    a1 = create_node(input_position, input_velocity, "a1", trainable_list[0], trainable_list[1], act_dict[x[0]])
    a2 = create_node(input_position, input_velocity, "a2", trainable_list[2], trainable_list[3], act_dict[x[1]])
    a3 = create_node(a1, a2, "a3", trainable_list[6], trainable_list[7], act_dict[x[2]])
    a4 = create_node(a1, a2, "a4", trainable_list[8], trainable_list[9], act_dict[x[3]])
    force_input = create_node(a1, a2, "fi", trainable_list[4], trainable_list[5], 'linear')

    #force at time t+delta_t/2 from halfstep position
    force1 = Dense(10, activation='tanh', use_bias = True, name='force_middle_layer', trainable=False)(force_input)
    force2 = Dense(10, activation='tanh', use_bias = True, name='force_middle_layer_2', trainable=False) (force1)
    force_output = Dense(1, activation='linear', use_bias = True, name='force_output_layer', trainable=False)(force2)

    #a5
    a51 = Dense(1, activation = 'linear', use_bias = False, name="a51", trainable=trainable_list[10]) (a3)
    a52 = Dense(1, activation = 'linear', use_bias = False, name="a52", trainable=trainable_list[11]) (force_output)
    a53 = Dense(1, activation = 'linear', use_bias = False, name="a53", trainable=trainable_list[12]) (a4)
    a5 = add([a51, a52, a53])
    a5 = Activation(act_dict[x[4]]) (a5)

    #a6
    a61 = Dense(1, activation = 'linear', use_bias = False, name="a61", trainable=trainable_list[13]) (a3)
    a62 = Dense(1, activation = 'linear', use_bias = False, name="a62", trainable=trainable_list[14]) (force_output)
    a63 = Dense(1, activation = 'linear', use_bias = False, name="a63", trainable=trainable_list[15]) (a4)
    a6 = add([a61, a62, a63])
    a6 = Activation(act_dict[x[5]]) (a6)
    
    #output_position
    output_position = create_node(a5, a6, "op", trainable_list[16], trainable_list[17], "linear")

    #output_velocity
    output_velocity = create_node(a5, a6, "ov", trainable_list[18], trainable_list[19], "linear")

    model = Model(inputs=[input_position, input_velocity], outputs=[output_position, output_velocity])
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
    model.compile(loss='mse', optimizer=optimizer)

    # FREEZING WEIGHTS
    model.layers[16].set_weights(w1)
    model.layers[20].set_weights(w2)
    model.layers[23].set_weights(w3)
    
    # SETTING OTHER WEIGHTS
    layer_list = []
    for i in range(2, 6):
        layer_list.append(i)
    for i in [10, 11, 14, 15, 17, 18]:
        layer_list.append(i)
    for i in range(25, 31):
        layer_list.append(i)
    for i in range(35, 39):
        layer_list.append(i)
    
    for i in range(len(layer_list)):
        model.layers[layer_list[i]].set_weights( [ np.array( [[ weight_dict[x[nact_terms+i]] ]] ) ] )

    #model.summary()

    return model, trainable_list, layer_list

We now define the training protocol for the network. Note that training is driven by the genetic algorithm, where the individual's genes decide whether a particular weight is fixed or trainable. If all the weights in the model are fixed, then the model is not trained

In [ ]:
losses = []
class PrintEpNum(keras.callbacks.Callback): # This is a function for the Epoch Counter
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + ' Loss: ' + str(logs.get('loss')) + '\n') # Updates current Epoch Number
        losses.append(logs.get('loss'))

def train(model, train_inputs, train_outputs, val_inputs, val_outputs, verbose=False):
    mae_es= keras.callbacks.EarlyStopping(monitor='val_loss', patience=100,
                                          min_delta=1e-25, verbose=1, mode='auto', restore_best_weights=True)

    EPOCHS = 20000 # Number of EPOCHS
    history = model.fit([train_inputs[:,0], train_inputs[:,1]], [train_outputs[:,0], train_outputs[:,1]], epochs=EPOCHS,
                        shuffle=False, batch_size=len(train_inputs), verbose = False, callbacks=[mae_es],
                        validation_data = ([val_inputs[:,0], val_inputs[:,1]], [val_outputs[:,0], val_outputs[:,1]]))

    if verbose:
        plt.figure()
        plt.xlabel('Epoch')
        plt.ylabel('Mean Sq Error')
        plt.plot(history.epoch, np.array(history.history['loss']),label='Training loss')
        plt.legend()
        plt.show()
    return

In [ ]:
def f3(w):
    if (w == 0):
        return 0
    elif (w >= 1 and w <= 6):
        return 1
    elif (w == 7):
        return 2

## Step 3: Define the objective function for the genetic algorithm
The objective function consists of three parts: 
1. The mean squared error of the model on the test set 
2. A penalty term for non-linear activation functions
3. A penalty term for weights that are not fixed, simple values such as 0, 1 or a multiple of an expected weight, such as the timestep

In [ ]:
def objective_function(individual):
    new_model, trainable, layers = create_model(individual)
    #print ("Trainable: ", trainable)
    if (any(trainable) == True):
        train(new_model, train_inputs, train_outputs, val_inputs, val_outputs, verbose=False)
    print("Model weights")
    for i in layers:
        print (new_model.layers[i].get_weights()[0])

    loss, mse_test_x, mse_test_v = new_model.evaluate([test_inputs[:,0:1], test_inputs[:,1:2]],
                                                      [test_outputs[:,0:1], test_outputs[:,1:2]], verbose=0)
    actfunc_term = [i**2 for i in individual[:nact_terms]]
    weights = individual[nact_terms:]
    weight_term = 0
    for j in range(nweight_terms):
        weight_term += f3(weights[j])

    mse_test_term_x = 10*np.log10(mse_test_x)
    mse_test_term_v = 10*np.log10(mse_test_v)

    obj = mse_test_term_x + mse_test_term_v + np.sum(actfunc_term) + weight_term
    print ("Individual: ", individual, flush=True)
    print ("Objective function: ", mse_test_x, mse_test_v, np.sum(actfunc_term), weight_term, obj, flush=True)
    K.clear_session()
    tf.reset_default_graph()
    return (obj,)

## Step 4: Set up the genetic algorithm and saving the results

Each network is expressed as an individual of 26 genes, the genes representing the possible activations and weights. We thus define an individual to be a custom container, which is repeated to create a population. For details on this, please refer to the DEAP guide on setting up a genetic algorithm, which can be found [here](https://deap.readthedocs.io/en/master/)

In [ ]:
################### DEAP #####################
#create fitness class and individual class
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
#toolbox.register("map", futures.map)

def custom_initRepeat(container, func, max1, max2, n):
    func_list = []
    for i in range(n):
        if (i < nact_terms):
            func_list.append(func(0, max1))
        else:
            func_list.append(func(0, max2))
    return container(func_list[i] for i in range(n))

toolbox.register("create_individual", custom_initRepeat, creator.Individual, random.randint,
                 max1=3, max2=7, n=nact_terms+nweight_terms)
toolbox.register("population", tools.initRepeat, list, toolbox.create_individual)

def custom_mutation(individual, max1, max2, indpb):
    size = len(individual)
    for i in range(size):
        if random.random() < indpb:
            if (i < nact_terms):
                individual[i] = random.randint(0, max1)
            else:
                individual[i] = random.randint(0, max2)
    return individual,

We use the two point crossover method for mating two individuals, and perform a random mutation using the custom mutation function. We then define a population size of 200 and define the statistics that we wish to log in the output of the code. The stats object decides which quantities are saved to the logbook

In [ ]:
cxpb = 0.5
mutpb = 0.3
ngens = 50

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", custom_mutation, max1=3, max2=7, indpb=mutpb)
toolbox.register("select", tools.selTournament, tournsize=10)
toolbox.register("evaluate", objective_function)


random.seed(100000)
population = toolbox.population(n=200)

hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)
pop, logbook = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngens, stats=stats, halloffame=hof, verbose=True)